# TP : Analyse de sentiments dans les critiques de films

## Objectifs

1. Implémenter une manière simple de représenter des données textuelles
2. Implémenter un modèle d'apprentissage statistique basique
3. Utiliser ces représentations et ce modèle pour une tâche d'analyse de sentiments
4. Tenter d'améliorer les résultats avec des outils venus du traitement automatique du langage
5. Comparer les résultats avec une l'implémentation de Scikit-Learn, et avec d'autres méthodes de représentation ou d'apprentissage.

## Dépendances nécessaires

Pour les objectifs 4. et 5., on aura besoin des packages suivants:
- The Machine Learning API Scikit-learn : http://scikit-learn.org/stable/install.html
- The Natural Language Toolkit : http://www.nltk.org/install.html

Les deux sont disponibles avec Anaconda: https://anaconda.org/anaconda/nltk et https://anaconda.org/anaconda/scikit-learn

In [31]:
import os.path as op
import re 
import numpy as np

## Charger les données

Extraire les données et placer le dossier 'data' dans le même dossier que le notebook.

On récupère les données textuelles dans la variable *texts*

On récupère les labels dans la variable $y$ qui en contient *len(texts)* : $0$ indique que la critique correspondante est négative tandis que $1$ qu'elle est positive.

In [32]:
from glob import glob
# We get the files from the path: ./data/imdb1/neg for negative reviews, and ./data/imdb1/pos for positive reviews
filenames_neg = sorted(glob(op.join('.', 'data', 'imdb1', 'neg', '*.txt')))
filenames_pos = sorted(glob(op.join('.', 'data', 'imdb1', 'pos', '*.txt')))

# Each files contains a review that consists in one line of text: we put this string in two lists, that we concatenate
texts_neg = [open(f, encoding="utf8").read() for f in filenames_neg]
texts_pos = [open(f, encoding="utf8").read() for f in filenames_pos]
texts = texts_neg + texts_pos

# The first half of the elements of the list are string of negative reviews, and the second half positive ones
# We create the labels, as an array of [1,len(texts)], filled with 1, and change the first half to 0
y = np.ones(len(texts), dtype=np.int)
y[:len(texts_neg)] = 0.

print("%d documents" % len(texts))

25000 documents


In [33]:
print(texts[12500])
print(y[12500])

Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!
1


In [129]:
# This number of documents may be high for most computers: we can select a fraction of them (here, one in k)
# Use an even number to keep the same number of positive and negative reviews
k = 10
texts_reduced = texts[0::k]
y_reduced = y [0::k]

print('Nombre de documents:', len(texts_reduced))

Nombre de documents: 2500


In [130]:
y_reduced.shape[0] // 2

1250

In [127]:
y_reduced[y_reduced==0].shape[0]

1137

## Idée principale

On dispose d'une critique étant en fait une liste de mots $s = (w_1, ..., w_N)$, et l'on cherche à trouver la classe associée $c$ - qui dans notre cas, peut-être $c = 0$ ou $c = 1$. L'objectif est donc de trouver pour chaque critique $s$ la classe $\hat{c}$ maximisant la probabilité conditionelle **$P(c|s)$** : 

$$\hat{c} = \underset{c}{\mathrm{argmax}}\, P(c|s) = \underset{c}{\mathrm{argmax}}\,\frac{P(s|c)P(c)}{P(s)}$$

**Hypothèse : P(s) est constante pour chaque classe** :

$$\hat{c} = \underset{c}{\mathrm{argmax}}\,\frac{P(s|c)P(c)}{P(s)} = \underset{c}{\mathrm{argmax}}\,P(s|c)P(c)$$

**Hypothèse naïve : les différentes variables (mots) d'une critique sont indépendantes entre elles** : 

$$P(s|c) = P(w_1, ..., w_N|c)=\Pi_{i=1..N}P(w_i|c)$$

On va donc pouvoir se servir des critiques annotées à notre disposition pour **estimer les probabilités $P(w|c)$ pour chaque mot $w$ étant donné les deux classes $c$**. Ces critiques vont nous permettre d'apprendre à évaluer la "compatibilité" entre les mots et classes.

## Vue générale

### Entraînement: Estimer les probabilités

Pour chaque mot $w$ du vocabulaire $V$, $P(w|c)$ est le nombre d'occurences de $w$ dans une critique ayant pour classe $c$, divisé par le nombre total d'occurences dans $c$. Si on note $T(w,c)$ ce nombre d'occurences, on obtient:

$$P(w|c) = \text{Fréquence de }w\text{ dans }c = \frac{T(w,c)}{\sum_{w' \in V} T(w',c)}$$

### Test: Calcul des scores

Pour faciliter les calculs et éviter les erreurs d'*underflow* et d'approximation, on utilise le "log-sum trick", et on passe l'équation en log-probabilités : 

$$\hat{c} =  \underset{c}{\mathrm{argmax}}\, P(c|s) = \underset{c}{\mathrm{argmax}}\, \left[ \mathrm{log}(P(c)) + \sum_{i=1..N}log(P(w_i|c)) \right]$$

### Laplace smoothing (Lissage)

Un mot qui n'apparaît pas dans un document a une probabilité nulle: cela va poser problème avec le logarithme. On garde donc une toute petite partie de la masse de probabilité qu'on redistribue avec le *Laplace smoothing*: 

$$P(w|c) = \frac{T(w,c) + 1}{\sum_{w' \in V} (T(w',c) + 1)}$$

Il existe d'autre méthodes de lissage, en général adaptées à d'autres applications plus complexes. 

## Représentation adaptée des documents

Notre modèle statistique, comme la plupart des modèles appliqués aux données textuelles, utilise les comptes d'occurences de mots dans un document. Ainsi, une manière très pratique de représenter un document est d'utiliser un vecteur "Bag-of-Words" (BoW), contenant les comptes de chaque mot (indifférement de leur ordre d'apparition) dans le document. 

Si on considère l'ensemble de tous les mots apparaissant dans nos $T$ documents d'apprentissage, que l'on note $V$ (Vocabulaire), on peut créer **un index**, qui est une bijection associant à chaque mot $w$ un entier, qui sera sa position dans $V$. 

Ainsi, pour un document extrait d'un ensemble de documents contenant $|V|$ mots différents, une représentation BoW sera un vecteur de taille $|V|$, dont la valeur à l'indice d'un mot $w$ sera son nombre d'occurences dans le document. 

On peut utiliser la classe **CountVectorizer** de scikit-learn pour mieux comprendre:

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, ClassifierMixin

In [36]:
corpus = ['I walked down down the boulevard',
          'I walked down the avenue',
          'I ran down the boulevard',
          'I walk down the city',
          'I walk down the the avenue']
vectorizer = CountVectorizer()

Bow = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names())
Bow.toarray()

['avenue', 'boulevard', 'city', 'down', 'ran', 'the', 'walk', 'walked']


array([[0, 1, 0, 2, 0, 1, 0, 1],
       [1, 0, 0, 1, 0, 1, 0, 1],
       [0, 1, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 0, 1, 1, 0],
       [1, 0, 0, 1, 0, 2, 1, 0]])

On affiche d'abord la liste contenant les mots ordonnés selon leur indice (On note que les mots de 2 caractères ou moins ne sont pas pris en compte).

## Détail: entraînement

L'idée est d'extraire le nombre d'occurences $T(w,c)$ de chaque mot $w$ pour chaque classe $c$, ce qui permettra de calculer la matrice de probabilités conditionelles $\pmb{P}$ telle que: $$\pmb{P}_{w,c} = P(w|c)$$

Notons que le nombre d'occurences $T(w,c)$ peut être obtenu facilement à partir des représentations BoW de l'ensemble des documents.

### Procédure:

- Extraire le vocabulaire $V$ et les comptes $T(w,c)$ pour chaque mot $w$ et chaque classe $c$, à partir d'un ensemble de documents.
- Calculer les probabilités à priori des classes $P(c) = \frac{\sum_{w \in V} T(w,c)}{\sum_{c \in C} \sum_{w \in V} T(w,c)}$
- Calculer la probabilité **lissée** probabilities $P(w|c) = \frac{T(w,c) + 1}{\sum_{w' \in V} T(w',c) + 1}$.

## Détail: test

Nous connaissons maintenant les probabilités conditionelles données par la matrice $\pmb{P}$. 
Il faut maintenant obtenir $P(s|c)$ pour le document courant. Cette quantité s'obtient à l'aide d'un calcul simple impliquant la représentation BoW du document et $\pmb{P}$.

### Procédure:
- For each of the classes $c$,
    - $Score(c) = \log P(c)$
    - For each word $w$ in the document to be tested:
        - $Score(c) += \log P(w|c)$
- Return $argmax_{c \in C} Score(c)$ 

## Preprocessing du texte: obtenir les représentations BoW

D'abord, il faut transformer les critiques sous forme de strings en une liste de mots. La tactique la plus simple consiste à diviser le string suivant les espaces, avec la commande:
```text.split()```

Mais il faut aussi faire attention à enlever les caractères particuliers qui pourraient ne pas avoir été nettoyés (comme les balises HTML si on a obtenu les données à partir de pages web). Puisque l'on va compter les mots, il faudra construire une liste des mots apparaissant dans nos données. Dans notre cas, on aimerait réduire cette liste et l'uniformiser (ignorer les majuscules, la ponctuation, et les mots les plus courts). 
On va donc utiliser une fonction adaptée à nos besoins - mais c'est un travail qu'il n'est en général pas nécessaire de faire, puisqu'il existe de nombreux outils déjà adaptés à la plupart des cas de figures. 
Pour le nettoyage du texte, il existe de nombreux scripts, basés sur différents outils (expressions régulières, par exemple) qui permettent de préparer des données. La division du texte en mots et la gestion de la ponctuation est gérée lors d'une étape appellée *tokenization*; si besoin, un package python comme le NLTK contient de nombreux *tokenizers* différents.

In [37]:
# We might want to clean the file with various strategies:
def clean_and_tokenize(text):
    """
    Cleaning a document with:
        - Lowercase        
        - Removing numbers with regular expressions
        - Removing punctuation with regular expressions
        - Removing other artifacts
    And separate the document into words by simply splitting at spaces
    Params:
        text (string): a sentence or a document
    Returns:
        tokens (list of strings): the list of tokens (word units) forming the document
    """        
    # Lowercase
    text = text.lower()
    # Remove numbers
    text = re.sub(r"[0-9]+", "", text)
    # Remove punctuation
    REMOVE_PUNCT = re.compile("[.;:!\'?,\"()\[\]]")
    text = REMOVE_PUNCT.sub("", text)
    # Remove small words (1 and 2 characters)
    text = re.sub(r"\b\w{1,2}\b", "", text)
    # Remove HTML artifacts specific to the corpus we're going to work with
    REPLACE_HTML = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    text = REPLACE_HTML.sub(" ", text)
    
    tokens = text.split()        
    return tokens

# Or we might want to use an already-implemented tool. The NLTK package has a lot of very useful text processing tools, among them various tokenizers
# Careful, NLTK was the first well-documented NLP package, but it might be outdated for some uses. Check the documentation !
from nltk.tokenize import word_tokenize

corpus_raw = "I walked down down the boulevard. I walked down the avenue. I ran down the boulevard. I walk down the city. I walk down the the avenue."
print(clean_and_tokenize(corpus_raw))
#print(word_tokenize(corpus_raw))

['walked', 'down', 'down', 'the', 'boulevard', 'walked', 'down', 'the', 'avenue', 'ran', 'down', 'the', 'boulevard', 'walk', 'down', 'the', 'city', 'walk', 'down', 'the', 'the', 'avenue']


Fonction **à compléter**. Elle prend en entrée une liste de document (chacun sous la forme d'un string) et renvoie, comme dans l'exemple utilisant ```CountVectorizer```:
- Un vocabulaire qui associe à chaque mot rencontré un index
- Une matrice, dont les lignes représentent les documents et les colonnes les mots indexés par le vocabulaire. En position $(i,j)$, on devra avoir le nombre d'occurence du mot $j$ dans le document $i$.

Le vocabulaire, qui était sous la forme d'une *liste* dans l'exemple précédent, pourra être renvoyé sous forme de *dictionnaire* dont les clés sont les mots et les valeurs les indices. Puisque le vocabulaire recense les mots du corpus sans se soucier de leur nombre d'occurences, on pourra le constituer à l'aide d'un ensemble (```set``` en python). 
On pourra bien sur utiliser la fonction ```clean_and_tokenize``` pour transformer les strings en liste de mots. 
##### Quelques pointeurs pour les débutants en Python : 

- ```my_list.append(value)``` : put the variable ```value``` at the end of the list ```my_list```

-  ```words = set()``` : create a set, which is a list of unique values 

- ```words.union(my_list)``` : extend the set ```words```

- ```dict(zip(keys, values)))``` : create a dictionnary

- ```for k, text in enumerate(texts)``` : syntax for a loop with the index, ```texts``` begin a list (of texts !)

- ```len(my-list)``` : length of the list ```my_list```


In [78]:
def count_words(texts):
    """Vectorize text : return count of each word in the text snippets

    Parameters
    ----------
    texts : list of str
        The texts
    Returns
    -------
    vocabulary : dict
        A dictionary that points to an index in counts for each word.
    counts : ndarray, shape (n_samples, n_features)
        The counts of each word in each text.
    """
    #
    # A compléter ! 
    #
    my_list = []
    words = set()
    for text in texts:
        split_text = clean_and_tokenize(text)
        my_list.append(split_text)
        words.update(split_text) 
        
    vocabulary = dict(zip(words, list(np.arange(len(words)))))
    
    range(1, len(words) + 1)
    
    n_samples, n_features = len(my_list), len(words)
    counts = np.zeros((n_samples, n_features))
    
    for i, text in enumerate(my_list):
        for word in text:
            counts[i][vocabulary[word]] += 1

    return vocabulary, counts

In [119]:
Voc, X = count_words(corpus)
print(Voc)
print(X)

{'the': 0, 'down': 1, 'ran': 2, 'walked': 3, 'walk': 4, 'city': 5, 'avenue': 6, 'boulevard': 7}
[[1. 2. 0. 1. 0. 0. 0. 1.]
 [1. 1. 0. 1. 0. 0. 1. 0.]
 [1. 1. 1. 0. 0. 0. 0. 1.]
 [1. 1. 0. 0. 1. 1. 0. 0.]
 [2. 1. 0. 0. 1. 0. 1. 0.]]


## Naïve Bayes 

Classe vide : fonctions **à compléter** 
```python
def fit(self, X, y)
``` 
**Entraînement** : va apprendre un modèle statistique basés sur les représentations $X$ correspondant aux labels $y$.
$X$ représente donc ici des représentations obtenues en sortie de count_words. On complète la fonction à l'aide de la procédure détaillée plus haut. Si il est possible de la suivre à la lettre, les représentations que l'on utilise nous permettre d'être bien plus efficace et d'éviter d'utiliser des boucles !


Note: le lissage, effectué à la ligne $10$, ne se fait pas nécessairement avec un $1$, mais peut se faire avec une valeur positive $\alpha$, qu'on peut implémenter comme argument de la classe ```NB```.

```python
def predict(self, X)
```
**Testing** : va renvoyer les labels prédits par le modèle pour d'autres représentations $X$.



Pour faciliter la procédure, on prendra la moitié de la matrice $X$ obtenue plus haut pour entraîner le modèle, et l'autre moitié pour l'évaluer. **Important**: cette façon de procéder n'est pas réaliste: en général, on ne dispose que des données d'entraînement au moment de créer le vocabulaire et d'entraîner le modèle. Ainsi, il est possible que les données d'évaluation contiennent des mots *inconnus*. C'est quelque chose qu'on peut traiter facilement en dédiant un indice à tous les mots rencontrés qui ne sont pas contenus dans le vocabulaire - mais il existe de nombreuses méthodes plus complexes pour réussir à utiliser à bon escient ces mots que le modèle n'a pas rencontré à l'entraînement. 

##### Quelques pointeurs pour les débutants en Python : 

Utiliser l'API Numpy pour travailler avec des tenseurs


- ```X.shape``` : for a ```numpy.array```, return the dimension of the tensor

- ```np.zeros((dim_1, dim_2,...))``` : create a tensor filled with zeros

- ```np.sum(X, axis = n)``` : sum the tensor over the axis n

- ```np.mean(X, axis = n)```

- ```np.argmax(X, axis = n)```

- ```np.log(X)```

- ```np.dot(X_1, X_1)``` : Matrix multiplication

In [136]:
X[:X.shape[0]//2].shape

(1250, 33343)

In [ ]:
class NB(BaseEstimator, ClassifierMixin):
    # Les arguments de classe permettent l'héritage de classes de sklearn
    def __init__(self, alpha=1.0):
        # alpha est un paramètre pour le lissage: il correspond à la valeur ligne 10 de l'algorithme d'entraînement
        # Dans l'algorithme d'entraînement, et comme valeur par défaut, on utilise alpha = 1
        self.alpha = alpha

    def fit(self, X, y):
        scores_c = np.zeros(2)
        scores_c[0] = np.log(y[y == 0].shape[0] / y.shape[0])
        scores_c[1] = np.log(y[y == 1].shape[0] / y.shape[0])
        
        
        
        
         
            
        
        return self

    def predict(self, X):
        #
        # A compléter ! 
        #
        return result

    def score(self, X, y):
        return np.mean(self.predict(X) == y)

## Expérimentation

On utilise la moitié des données pour l'entraînement, l'autre pour tester le modèle.

In [132]:
voc, X = count_words(texts_reduced)

In [ ]:
nb = NB()
nb.fit(X[::2], y_reduced[::2])
print(nb.score(X[1::2], y_reduced[1::2]))

## Cross-validation 

Avec la fonction *cross_val_score* de scikit-learn

In [ ]:
scores = cross_val_score(nb, X, y_reduced, cv=5)
print('Score de classification: %s (std %s)' % (np.mean(scores), np.std(scores)))

## Evaluer les performances: 

**Quelles sont les points forts et les points faibles de ce système ? Comment y remédier ?**

# Pour aller plus loin: 

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

## Scikit-learn

### Améliorer les représentations

On utilise la function 
```python
CountVectorizer
``` 
de scikit-learn pour constituer notre corpus. Elle nous permettra d'améliorer facilement nos représentations BoW.

#### Tf-idf:

Il s'agit du produit de la fréquence du terme (TF) et de sa fréquence inverse dans les documents (IDF).
Cette méthode est habituellement utilisée pour extraire l'importance d'un terme $i$ dans un document $j$ relativement au reste du corpus, à partir d'une matrice d'occurences $mots \times documents$. Ainsi, pour une matrice $\mathbf{T}$ de $|V|$ termes et $D$ documents:
$$\text{TF}(T, w, d) = \frac{T_{w,d}}{\sum_{w'=1}^{|V|} T_{w',d}} $$

$$\text{IDF}(T, w) = \log\left(\frac{D}{|\{d : T_{w,d} > 0\}|}\right)$$

$$\text{TF-IDF}(T, w, d) = \text{TF}(X, w, d) \cdot \text{IDF}(T, w)$$

On peut l'adapter à notre cas en considérant que le contexte du deuxième mot est le document. Cependant, TF-IDF est généralement plus adaptée aux matrices peu denses, puisque cette mesure pénalisera les termes qui apparaissent dans une grande partie des documents. 
    
#### Ne pas prendre en compte les mots trop fréquents:

On peut utiliser l'option 
```python
max_df=1.0
```
pour modifier la quantité de mots pris en compte. 

#### Essayer différentes granularités:

Plutôt que de simplement compter les mots, on peut compter les séquences de mots - de taille limitée, bien sur. 
On appelle une séquence de $n$ mots un $n$-gram: essayons d'utiliser les 2 et 3-grams (bi- et trigrams).
On peut aussi tenter d'utiliser les séquences de caractères à la place de séquences de mots.

On s'intéressera aux options 
```python
analyzer='word'
```
et 
```python
ngram_range=(1, 2)
```
que l'on changera pour modifier la granularité. 

In [ ]:
## On peut définir une pipeline que l'on modifiera pour expérimenter.

pipeline_base = Pipeline([
    ('vect', CountVectorizer(analyzer='word', stop_words=None)),
    ('clf', MultinomialNB()),
])
scores = cross_val_score(pipeline_base, texts_reduced, y_reduced, cv=5)
print("Classification score: %s (std %s)",(np.mean(scores), np.std(scores)))

pipeline_tf_idf = Pipeline([
        #
        # A compléter ! 
        #
])
scores = cross_val_score(pipeline_tf_idf, texts_reduced, y_reduced, cv=5)
print("Classification score tf-idf: %s (std %s)",(np.mean(scores), np.std(scores)))

pipeline_maxdf = Pipeline([
        #
        # A compléter ! 
        #
])
scores = cross_val_score(pipeline_maxdf, texts_reduced, y_reduced, cv=5)
print("Classification score sans mots fréquents: %s (std %s)",(np.mean(scores), np.std(scores)))

pipeline_bigram = Pipeline([
        #
        # A compléter ! 
        #
])
scores = cross_val_score(pipeline_bigram, texts_reduced, y_reduced, cv=5)
print("Classification score bigram: %s (std %s)",(np.mean(scores), np.std(scores)))

pipeline_trigram = Pipeline([
        #
        # A compléter ! 
        #
])
scores = cross_val_score(pipeline_trigram, texts_reduced, y_reduced, cv=5)
print("Classification score trigram: %s (std %s)",(np.mean(scores), np.std(scores)))

pipeline_char = Pipeline([
        #
        # A compléter ! 
        #
])
scores = cross_val_score(pipeline_char, texts_reduced, y_reduced, cv=5)
print("Classification score char: %s (std %s)",(np.mean(scores), np.std(scores)))

### Natural Language Toolkit (NLTK)

### Stemming 

Permet de revenir à la racine d'un mot: on peut ainsi grouper différents mots autour de la même racine, ce qui facilite la généralisation. Utiliser:
```python
from nltk import SnowballStemmer
```

In [ ]:
from nltk import SnowballStemmer
stemmer = SnowballStemmer("english")

#### Exemple d'utilisation:

In [ ]:
words = ['singers', 'cat', 'generalization', 'philosophy', 'psychology', 'philosopher']
for word in words:
    print('word : %s ; stemmed : %s' %(word, stemmer.stem(word)))#.decode('utf-8'))))

#### Transformation des données:

Classe vide : function **à compléter** 
```python
def stem(X)
``` 

In [ ]:
def stem(X): 
    X_stem = []
    #
    # A compléter ! 
    #
    return X_stem

In [ ]:
texts_stemmed = stem(texts_reduced)
voc, X = count_words(texts_stemmed)
nb = NB()

scores = cross_val_score(nb, X, y_reduced, cv=5)
print('Score de classification: %s (std %s)' % (np.mean(scores), np.std(scores)))

### Partie du discours

Pour généraliser, on peut aussi utiliser les parties du discours (Part of Speech, POS) des mots, ce qui nous permettra  de filtrer l'information qui n'est potentiellement pas utile au modèle. On va récupérer les POS des mots à l'aide des fonctions:
```python
from nltk import pos_tag, word_tokenize
```

In [ ]:
import nltk
from nltk import pos_tag, word_tokenize

#### Exemple d'utilisation:

In [ ]:
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

pos_tag(word_tokenize(('I am Sam')))

Détails des significations des tags POS: https://stackoverflow.com/questions/15388831/what-are-all-possible-pos-tags-of-nltk

####  Transformation des données:

Classe vide : fonction **à compléter** 
```python
def pos_tag_filter(X, good_tags=['NN', 'VB', 'ADJ', 'RB'])
``` 

Ne garder que les noms, adverbes, verbes et adjectifs pour notre modèle. 

In [ ]:
def pos_tag_filter(X, good_tags=['NN', 'VB', 'ADJ', 'RB']):
    X_pos = []
    #
    # A compléter ! 
    #
    return X_pos

In [ ]:
texts_POS = pos_tag_filter(texts_reduced)
voc, X = count_words(texts_POS)
nb = NB()

scores = cross_val_score(nb, X, y_reduced, cv=5)
print('Score de classification: %s (std %s)' % (np.mean(scores), np.std(scores)))

### Bonus: Utilisation d'un classifieur plus complexe ?

On peut utiliser les implémentations scikit-learn de classifieurs moins naïfs, comme la régression logistique ou les SVM. 

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

In [ ]:
pipeline_logistic = Pipeline([
    #
    # A compléter ! 
    #
])
scores = cross_val_score(pipeline_logistic, texts_reduced, y_reduced, cv=5)
print("Classification score: %s (std %s)" % (np.mean(scores), np.std(scores)))

pipeline_svm = Pipeline([
    #
    # A compléter ! 
    #
])
scores = cross_val_score(pipeline_svm, texts_reduced, y_reduced, cv=5)
print("Classification score: %s (std %s)" % (np.mean(scores), np.std(scores)))